# Obtaining metrics

## Dataset: Intel

## Output:

- LSA values
    ```python
    "C:/Users/fjdur/Desktop/upc/project_notebooks/github_project/DL_notebooks/SA_values/intel_lsa_values_2.npy"
    ```
- DSA values
    ```python
    "C:/Users/fjdur/Desktop/upc/project_notebooks/github_project/DL_notebooks/SA_values/intel_dsa_values_2.npy"
    ```
- Random values
    ```python
    "C:/Users/fjdur/Desktop/upc/project_notebooks/github_project/DL_notebooks/SA_values/intel_random_values_e2.npy"
    ```
- NC values
    ```python
    'intel_nc_values_'+str(i)+'.npy'
    ```


In [4]:
import argparse

import numpy as np
import tensorflow as tf
import keras.backend as K

import matplotlib.pyplot as plt

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.regularizers import l2

import glob
import os
import cv2
import keras


import time
import argparse

from tqdm import tqdm

from keras.models import load_model, Model


In [5]:
# Params
num_classes = 6
img_width = 48
img_height = 48

In [ ]:
cd '../utils/'

In [ ]:
# utils for project
import utils_guided_retraining as utils

In [ ]:
cd '../notebooks/'

In [ ]:
x_train,y_train = utils.get_data("intel","Train")
x_val,y_val = utils.get_data("intel","Val")
x_test,y_test = utils.get_data("intel","Test")

In [ ]:
x_train_and_adversary,y_train_and_adversary = utils.get_data("intel","Train_and_adversary")

In [ ]:
x_adversary_training = x_train_and_adversary[len(x_train):]
print(len(x_adversary_training))
y_adversary_training = y_train_and_adversary[len(y_train):]

print(len(y_adversary_training))

In [ ]:
# Obtaining adversarial examples for testing 
x_test_and_adversary,y_test_and_adversary = utils.get_adversarial_data("intel",'Test_fgsm') 

In [ ]:
x_adversary_test_fgsm = x_test_and_adversary[len(x_test):]
print(len(x_adversary_test_fgsm))
y_adversary_test_fgsm = y_test_and_adversary[len(y_test):]

print(len(y_adversary_test_fgsm))

## loading models

In [16]:
# Model compiled without other metrics
model_dir = "C:/Users/fjdur/Desktop/upc/project_notebooks/github_project/DL_notebooks/models/intel_model_21_10/"
#model_dir = "C:/Users/fjdur/Desktop/upc/project_notebooks/github_project/DL_notebooks/models/intel_model"
model = keras.models.load_model(model_dir)

In [18]:
model.evaluate(x_test,y_test)

94/94 [==============================] - 12s 21ms/step - loss: 0.5392 - accuracy: 0.8037


[0.5391883850097656, 0.8036666512489319]

In [52]:
print(x_train_and_adversary.shape)
print(x_test_and_adversary.shape)

(14224, 48, 48, 1)
(6000, 48, 48, 1)


## Obtaining new LSA and DSA values 
- Required module 'deepimportance_code_release'

In [53]:
cd "C:\Users\fjdur\Desktop\upc\project_notebooks\deepimportance_code_release"

C:\Users\fjdur\Desktop\upc\project_notebooks\deepimportance_code_release


In [54]:
# from 07_notebook
from utils import filter_val_set, get_trainable_layers
from utils import generate_adversarial, filter_correct_classifications
from coverages.idc import ImportanceDrivenCoverage
from coverages.neuron_cov import NeuronCoverage
from coverages.tkn import DeepGaugeLayerLevelCoverage
from coverages.kmn import DeepGaugePercentCoverage
from coverages.ss import SSCover
from coverages.sa import SurpriseAdequacy


In [55]:
trainable_layers = get_trainable_layers(model)
non_trainable_layers = list(set(range(len(model.layers))) - set(trainable_layers))
print('Trainable layers: ' + str(trainable_layers))
print('Non trainable layers: ' + str(non_trainable_layers))

Trainable layers: [0, 2, 4, 7, 12, 14, 15]
Non trainable layers: [1, 3, 5, 6, 8, 9, 10, 11, 13]


In [56]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 128)         0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 6, 6, 128)        

In [57]:
subject_layer = trainable_layers[-2]
layer_names = [model.layers[-2].name]
print(layer_names)
upper_bound = 2000

['dense_1']


In [59]:
from coverages import sa
from datetime import datetime

In [60]:
start_time = datetime.now()



target_lsa = sa.fetch_lsa(model, x_train, x_train_and_adversary,
                                   "intel_x_train_and_adversary_fgsm_2", layer_names,
                                   num_classes, True,
                                   1e-05, './sa_data', "intel_model1_2")


end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

[train] Model serving
16
88/88 [==============================] - 7s 76ms/step
[train] Processing ATs
Layer: dense_1
train ATs is saved at ./sa_data\intel_model1_2_train_dense_1_ats.npy
[intel_x_train_and_adversary_fgsm_2] Model serving
16
112/112 [==============================] - 10s 85ms/step
[intel_x_train_and_adversary_fgsm_2] Processing ATs
Layer: dense_1
intel_x_train_and_adversary_fgsm_2 ATs is saved at ./sa_data\intel_model1_2_intel_x_train_and_adversary_fgsm_2_dense_1_ats.npy
yes
dict_keys([3, 0, 5, 4, 1, 2])
The number of removed columns: 33
[intel_x_train_and_adversary_fgsm_2] Fetching LSA
Duration: 0:00:57.484108


In [61]:
start_time = datetime.now()

target_dsa = sa.fetch_dsa(model, x_train, x_train_and_adversary,
                                   "intel_x_train_and_adversary_fgsm_2", layer_names,
                                   num_classes, True,
                                   './sa_data',  "intel_model1_2")
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

Found saved train ATs, skip serving
[intel_x_train_and_adversary_fgsm_2] Model serving
16
112/112 [==============================] - 9s 79ms/step
[intel_x_train_and_adversary_fgsm_2] Processing ATs
Layer: dense_1
intel_x_train_and_adversary_fgsm_2 ATs is saved at ./sa_data\intel_model1_2_intel_x_train_and_adversary_fgsm_2_dense_1_ats.npy
[intel_x_train_and_adversary_fgsm_2] Fetching DSA
Duration: 0:02:04.818284


In [62]:
print(sum(target_lsa)/len(target_lsa))
print(len(target_lsa))

-141.29960938911742
14224


In [63]:
target_lsa[len(x_train):len(x_train)+10]

[-96.74262316765555,
 -89.78846137782241,
 -109.34962825007679,
 -145.2803882902978,
 -74.22917489798564,
 -124.4235144192323,
 -111.69370484500891,
 -114.96432152881017,
 -114.99649412355818,
 -114.13783385864635]

In [64]:
print(sum(target_dsa)/len(target_dsa))
print(len(target_dsa))

0.11192677398003051
14224


In [65]:
save_dir = "C:/Users/fjdur/Desktop/upc/project_notebooks/github_project/DL_notebooks/SA_values/intel_dsa_values_2.npy"

np.save(save_dir,np.array(target_dsa))
save_dir = "C:/Users/fjdur/Desktop/upc/project_notebooks/github_project/DL_notebooks/SA_values/intel_lsa_values_2.npy"

np.save(save_dir,np.array(target_lsa))

## Obtaining NC values 
- Required module 'TestSelection'

In [1]:
cd C:\Users\fjdur\Desktop\upc\project_notebooks

C:\Users\fjdur\Desktop\upc\project_notebooks


In [2]:
#import sys
#sys.path.append('C:/Users/Francisco Duran/Desktop/upc/project_notebooks')
import TestSelection
from TestSelection.utils.ncoverage import NCoverage # https://github.com/TestSelection/TestSelection

In [23]:
# Original model 
model_dir = "C:/Users/fjdur/Desktop/upc/project_notebooks/github_project/DL_notebooks/models/intel_model_21_10/"

model = utils.My_model("intel",True, model_dir)

Model loaded correctly


In [24]:
model.model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 128)         0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 6, 6, 128)        

In [22]:
# NC computor, one layer, model_1
ncComputor_model_1 = NCoverage(model, threshold=0.2,only_layers=[ 'dense_1']) #last hidden layer

models loaded
['dense_1']


In [25]:
# NC computor, one layer, model_1
ncComputor_model_1 = NCoverage(model.model, threshold=0.2,only_layers=[ 'dense_1']) #last hidden layer

models loaded
['dense_1']


In [ ]:
cd '../utils/'

In [7]:
# once obtained the adversarial data

import utils_guided_retraining as utils

In [ ]:
cd '../notebooks/'

In [26]:
x_train_and_adversary,y_train_and_adversary = utils.get_data("intel","Train_and_adversary")

In [27]:
len(x_train_and_adversary)

14224

In [28]:
x_adversary_training = x_train_and_adversary[11224:]
print(len(x_adversary_training))
y_adversary_training = y_train_and_adversary[11224:]

print(len(y_adversary_training))

3000
3000


In [ ]:
nc_model_original_values_1 = [ncComputor_model_1.NC(tf.expand_dims(x, axis=0)) for x in x_train_and_adversary[:1000]]

In [30]:
import time
start_time = time.time()
nc_model_original_values_1 = [ncComputor_model_1.NC(tf.expand_dims(x, axis=0)) for x in x_train_and_adversary[:1000]]
print("--- %s seconds ---" % (time.time() - start_time))


--- 297.05143308639526 seconds ---


In [34]:
start_time = time.time()
nc_model_original_values_1 = [ncComputor_model_1.NC(tf.expand_dims(x, axis=0)) for x in x_train_and_adversary[:1000]]
print("--- %s seconds ---" % (time.time() - start_time))

--- 279.7343292236328 seconds ---


In [28]:
nc_model_original_values_2 = [ncComputor_model_1.NC(tf.expand_dims(x, axis=0)) for x in x_train_and_adversary[1000:2000]]

In [33]:

start_time = time.time()
nc_model_original_values_2 = [ncComputor_model_1.NC(tf.expand_dims(x, axis=0)) for x in x_train_and_adversary[1000:2000]]
print("--- %s seconds ---" % (time.time() - start_time))



--- 287.054758310318 seconds ---


In [32]:

start_time = time.time()
nc_model_original_values_14 = [ncComputor_model_1.NC(tf.expand_dims(x, axis=0)) for x in x_train_and_adversary[13000:]]
print("--- %s seconds ---" % (time.time() - start_time))



--- 367.07521963119507 seconds ---


In [29]:
nc_model_original_values_3 = [ncComputor_model_1.NC(tf.expand_dims(x, axis=0)) for x in x_train_and_adversary[2000:3000]]

In [35]:
start_time = time.time()
nc_model_original_values_3 = [ncComputor_model_1.NC(tf.expand_dims(x, axis=0)) for x in x_train_and_adversary[2000:3000]]
print("--- %s seconds ---" % (time.time() - start_time))



--- 287.0088303089142 seconds ---


In [30]:
nc_model_original_values_4 = [ncComputor_model_1.NC(tf.expand_dims(x, axis=0)) for x in x_train_and_adversary[3000:4000]]
nc_model_original_values_5 = [ncComputor_model_1.NC(tf.expand_dims(x, axis=0)) for x in x_train_and_adversary[4000:5000]]

In [31]:
nc_model_original_values_6 = [ncComputor_model_1.NC(tf.expand_dims(x, axis=0)) for x in x_train_and_adversary[5000:6000]]


In [32]:
nc_model_original_values_7 = [ncComputor_model_1.NC(tf.expand_dims(x, axis=0)) for x in x_train_and_adversary[6000:7000]]
nc_model_original_values_8 = [ncComputor_model_1.NC(tf.expand_dims(x, axis=0)) for x in x_train_and_adversary[7000:8000]]

In [33]:
nc_model_original_values_9 = [ncComputor_model_1.NC(tf.expand_dims(x, axis=0)) for x in x_train_and_adversary[8000:9000]]
nc_model_original_values_10 = [ncComputor_model_1.NC(tf.expand_dims(x, axis=0)) for x in x_train_and_adversary[9000:10000]]
nc_model_original_values_11 = [ncComputor_model_1.NC(tf.expand_dims(x, axis=0)) for x in x_train_and_adversary[10000:11000]]

In [34]:
nc_model_original_values_12 = [ncComputor_model_1.NC(tf.expand_dims(x, axis=0)) for x in x_train_and_adversary[11000:12000]]


In [35]:
nc_model_original_values_13 = [ncComputor_model_1.NC(tf.expand_dims(x, axis=0)) for x in x_train_and_adversary[12000:13000]]
nc_model_original_values_14 = [ncComputor_model_1.NC(tf.expand_dims(x, axis=0)) for x in x_train_and_adversary[13000:]]

In [36]:
cd C:\Users\fjdur\Desktop\upc\project_notebooks\github_project\DL_notebooks\NC_values

C:\Users\fjdur\Desktop\upc\project_notebooks\github_project\DL_notebooks\NC_values


In [37]:
nc_values = [
    nc_model_original_values_1,
    nc_model_original_values_2,
    nc_model_original_values_3,
    nc_model_original_values_4,
    nc_model_original_values_5,
    nc_model_original_values_6,
    nc_model_original_values_7,
    nc_model_original_values_8,
    nc_model_original_values_9,
    nc_model_original_values_10,
    nc_model_original_values_11,
    nc_model_original_values_12,
    nc_model_original_values_13,
    nc_model_original_values_14  
]

In [38]:
#Save nc values
i=1
for values in nc_values:
    with open('intel_nc_values_'+str(i)+'.npy', 'wb') as f:
        np.save(f, np.array(values))
    i+=1

## Random values

In [ ]:
import random
random_indexes =list(range(len(x_train_and_adversary)))
random.shuffle(random_indexes)
print(random_indexes[:10])
print(len(random_indexes))

In [ ]:
save_dir = "C:/Users/fjdur/Desktop/upc/project_notebooks/github_project/DL_notebooks/SA_values/intel_random_values_e2.npy"

np.save(save_dir,np.array(random_indexes))

#random_indexes = np.load(save_dir)